<a href="https://colab.research.google.com/github/JoyeBright/Multilingual_Multimodal_Universal_MT/blob/main/OpenNMT_GRU_ManythingsDataset_Per_Eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Install OpenNMT**

In [26]:
!pip install --upgrade pip
!pip install OpenNMT-tf

     |████████████████████████████████| 1.5 MB 6.7 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 20.3.4
    Uninstalling pip-20.3.4:
      Successfully uninstalled pip-20.3.4


### **OpenNMT version**

In [27]:
!onmt-main -v

2021-01-23 20:44:40.413167: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
OpenNMT-tf 2.14.0


### **Preparing Dataset**

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
!wget "https://www.manythings.org/anki/pes-eng.zip"

--2021-01-23 17:06:03--  https://www.manythings.org/anki/pes-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3036::ac43:adc6, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.55.222|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 112553 (110K) [application/zip]
Saving to: ‘pes-eng.zip’

pes-eng.zip         100%[===================>] 109.92K   527KB/s    in 0.2s    

2021-01-23 17:06:04 (527 KB/s) - ‘pes-eng.zip’ saved [112553/112553]



In [6]:
!unzip pes-eng.zip -d "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data"

Archive:  pes-eng.zip
  inflating: /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/_about.txt  
  inflating: /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/pes.txt  


In [7]:
path = "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/pes.txt"

In [8]:
f = open(path, "r")
print(f.read(978)) 

Who?	چه کسی؟	CC-BY 2.0 (France) Attribution: tatoeba.org #2083030 (CK) & #4746184 (mahdiye)
Go on.	ادامه بده ( ادامه دادن )	CC-BY 2.0 (France) Attribution: tatoeba.org #2230774 (CK) & #6669169 (arashorosia)
Smile.	لبخند بزن.	CC-BY 2.0 (France) Attribution: tatoeba.org #2764108 (CK) & #4746196 (mahdiye)
Attack!	حمله!	CC-BY 2.0 (France) Attribution: tatoeba.org #1972610 (CK) & #4746181 (mahdiye)
Got it!	گرفتم!	CC-BY 2.0 (France) Attribution: tatoeba.org #320484 (CM) & #888645 (mahdiye)
I know.	من می دانم.	CC-BY 2.0 (France) Attribution: tatoeba.org #319990 (CK) & #888643 (mahdiye)
Listen.	گوش کن.	CC-BY 2.0 (France) Attribution: tatoeba.org #1913088 (CK) & #4746178 (mahdiye)
Really?	واقعا؟	CC-BY 2.0 (France) Attribution: tatoeba.org #373216 (kotobaboke) & #5118169 (Hussein64)
Really?	جدا؟	CC-BY 2.0 (France) Attribution: tatoeba.org #373216 (kotobaboke) & #5118171 (Hussein64)
Why me?	چرا من؟	CC-BY 2.0 (France) Attribution: tatoeba.org #24958 (CK) & #4746261 (mahdiye)



In [9]:
import re

In [10]:
text_file = open("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/English.txt", "w")
text_file2 = open("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/Persian.txt", "w")

f = open(path, "r")
for r in f:
  text_file.write(re.split(r'\t+', r)[0]+"\n") # English txt
  text_file2.write(re.split(r'\t+', r)[1]+"\n") # Persian txt

In [11]:
text_file.close()
text_file2.close()

### **Dataset Splits: Train, Validation, Test**

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
import os

In [13]:
English = np.genfromtxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/English.txt", dtype='str', delimiter="\n")
Persian = np.genfromtxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/Persian.txt", dtype='str', delimiter="\n")

In [14]:
print(English[:10]) #target
print(Persian[:10]) #source

['Who?' 'Go on.' 'Smile.' 'Attack!' 'Got it!' 'I know.' 'Listen.'
 'Really?' 'Really?' 'Why me?']
['چه کسی؟' 'ادامه بده ( ادامه دادن )' 'لبخند بزن.' 'حمله!' 'گرفتم!'
 'من می دانم.' 'گوش کن.' 'واقعا؟' 'جدا؟' 'چرا من؟']


In [15]:
src_train, src_test, tgt_train, tgt_test = train_test_split(Persian, English, test_size=0.2, random_state=1)

src_train, src_val, tgt_train, tgt_val = train_test_split(src_train, tgt_train, test_size=0.25, random_state=1)

In [16]:
print("Source (Train, Test, Val):" , src_train.shape[0], src_test.shape[0], src_val.shape[0])
print("Target (Train, Test, Va)l:" , tgt_train.shape[0], tgt_test.shape[0], tgt_val.shape[0])

Source (Train, Test, Val): 1365 455 455
Target (Train, Test, Va)l: 1365 455 455


In [17]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-train.txt", src_train, encoding="UTF-8", fmt = '%s')

In [18]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-train.txt", tgt_train, encoding="UTF-8", fmt = '%s')

In [19]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-test.txt", src_test, encoding="UTF-8", fmt = '%s')

In [20]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-test.txt", tgt_test, encoding="UTF-8", fmt = '%s')

In [21]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-val.txt", src_val, encoding="UTF-8", fmt = '%s')

In [22]:
np.savetxt("/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-val.txt", tgt_val, encoding="UTF-8", fmt = '%s')

### **Creating Vocabulary**

In [29]:
!pip install sacremoses # used as an English tokenizer

### **Generate English vocabulary from raw training files with on-the-fly tokenization (online)**

In [30]:
!onmt-build-vocab --size 50000 --save_vocab /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/data/tgt-vocab.txt /content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/tgt-train.txt

2021-01-23 20:47:19.747617: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-01-23 20:47:22.068379: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-23 20:47:22.070279: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-01-23 20:47:22.081348: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-01-23 20:47:22.081421: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dced393d028e): /proc/driver/nvidia/version does not exist
2021-01-23 20:47:22.082034: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


### **Generate Persian vocabulary from customized tokenization**

In [32]:
from __future__ import unicode_literals
!pip install hazm
from hazm import *

In [33]:
def preprocess_persian_sentence(w):
  normalizer = Normalizer() # Persian normalizer instance (Hazm)
  w = normalizer.normalize(w)
  w = word_tokenize(w)
  return w

In [ ]:
import io

text_file = "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/src-train.txt"

tokenized_file = "/content/drive/MyDrive/Multilingual_Multimodal_Unversal_MT/Per_Eng_GRU_V1/raw_data/test.txt"

lines = io.open(text_file, encoding="UTF-8").read()

tokenized_array = []

for line in lines.splitlines():
  # print(preprocess_persian_sentence(line))
  # print(type(preprocess_persian_sentence(line)))
  # print(" ".join(str(x) for x in preprocess_persian_sentence(line)))
  tokenized_array.append(" ".join(str(x) for x in preprocess_persian_sentence(line)))

In [39]:
# print(tokenized_array)
with open(tokenized_file, 'w') as f:
    for item in tokenized_array:
        f.write("%s\n" % item)